<a href="https://colab.research.google.com/github/theysee/sberbank_reboot/blob/master/%D0%9F%D1%80%D0%BE%D0%BC%D0%B5%D0%B6%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%BE%D0%B5_%D1%82%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%A2%D0%B2%D0%BE%D1%80%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%A1%D0%B5%D1%80%D0%B3%D0%B5%D0%B9_%D0%90%D1%80%D1%82%D0%B5%D0%B5%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Творческое задание
## Розничные покупки онлайн
**Сюжет: Промышленный шпионаж**

К вам в руки попал дамп базы продаж одного из крупных онлайн ретейлеров. Этот ретейлер является вашим конкурентом, и вы начали исследовать полученные данные.

**Требования**

0.   Скопируйте исходный ноутбук себе. В своём ноутбуке делаете задание
1.   Гуглить разрешается
2.   Убирайте ненужные вам клетки с кодом, и постарайтесь не отходить от логической структуры этого ноутбука
3.   Оформляйте графики так, чтобы они были читаемыми. Оформляйте легенду, заголовки, подписывайте оси в соответствующем масштабе и единицах измерения
4.   **Ваш ноутбук должен запускаться, и выполняться без ошибок сверху вниз. Иначе творческая работа оценивается в 0 баллов.**
5.   После того как вы сделали работу, нужно вызвать ctrl+s (сохранить ноутбук),
затем File->Download as .ipynb. Полученный файл нужно залить в форму опросника в соответствующей секции. В названии ноутбука не забудьте указать **имя и фамилию** в соответствующем месте


. 

Всего баллов 48.

### Шаг 1. Импортируйте нужные библиотеки. (1 балл)

In [0]:
# ваш код тут

### Шаг 2. Загрузите данные в переменную online_rt (1 балл)
Замечание: если вы получаете ошибку декодирования, то передайте аргумент encoding='latin1' в pd.read_csv().

Файл расположен по адресу 'http://89.223.95.235/file/Online_Retail.csv'
его можно сразу передавать в функцию pd.read_csv

In [0]:
# ваш код тут

### Шаг 3. Удалите из выборки некорректные данные (2 балла)
Похоже, что у вашего конкурента не самая консистентная база данных, удалите некорректные строки из датафрейма

In [0]:
# ваш код тут

### Шаг 4.1. Отобразите столбчатую диаграмму с 10-ю странами в которых произошло наибольшее количество продаж в терминах количества (Quantity) (5 баллов)

In [0]:
# ваш код тут

### Шаг 4.2. Отобразите такую же диаграмму, но без Великобритании (1 балл)

In [0]:
# ваш код тут

### Шаг 5. Создаёте колонку выручки с каждой продажи (2 балла)

In [0]:
# ваш код тут

### Шаг 6.1. Отобразите график отображающий зависимость выручки от стоимости за единицу (для всего набора данных). (10 баллов)
График должен быть читаемым. Предлагается разбить UnitPrice на интервалы, в этом могут помочь функции np.arange() и pd.cut().

In [0]:
# ваш код тут

### Шаг 6.2 Какой вывод можно сделать из этого графика? (1 балл)

*Ваш комментарий:*

### Шаг 7.1. Создайте колонки day и month целочисленного типа. (5 баллов)
Подсказка: для начала определите в каком формате содержится InvoiceDate

In [0]:
# ваш код тут

### Шаг 7.2. Отобразите помесячную выручку для всех стран за 2011 год (по отдельности) в виде stacked barplot (10 баллов)

In [0]:
# ваш код тут

### Шаг 7.3. Отобразите такой же график, но без Великобритании (1 балл)

In [0]:
# ваш код тут

### Шаг 7.4. Какой вывод можно сделать из этих графиков? (1 балл)

*Ваш комментарий:*

### Шаг 8.1. Отобразите для Нидерландов график дневной выручки в Ноябре 2011. (5 баллов)

In [0]:
# ваш код тут

### Шаг 8.2. В какой день выручка максимальная? (2 балла)

In [0]:
# ваш код тут